0. 환경 초기화

In [ ]:
import pandas as pd
from scipy.stats import norm
import numpy as np

import folium
from folium import plugins as fp
from IPython import display as dp

import time
import os
path = os.getcwd()

In [ ]:
pd.set_option('display.max_rows', None)

1. 자료 불러오기

1-1. 역 별 인원

https://data.seoul.go.kr/dataList/OA-12252/S/1/datasetView.do
서울시에서 제공하는, 23년 12월 3일에 작업된 23년 11월까지의 csv 데이터이다.
다루기 용이하게 column명을 바꾸었다.

In [ ]:
locx1 = path+"\\1_columnchanged\metro2312.csv"

x1 = pd.read_csv(locx1, encoding='cp949')
df1 = pd.DataFrame(x1)

In [ ]:
gdf1 = df1.groupby(['name'])

# data 원본은 승차, 하차, 승차, 하차, ...
# 다루기 편하게 승차 column(i**), 하차 column(o**) 나눠놓음
mean = gdf1[['i04', 'i05', 'i06', 'i07', 'i08', 'i09', 'i10', 'i11', 'i12', 'i13', 'i14', 'i15', 'i16', 'i17', 'i18', 'i19', 'i20', 'i21', 'i22', 'i23', 'i00', 'i01', 'i02', 'i03', 'o04', 'o05', 'o06', 'o07', 'o08', 'o09', 'o10', 'o11', 'o12', 'o13', 'o14', 'o15', 'o16', 'o17', 'o18', 'o19', 'o20', 'o21', 'o22', 'o23', 'o00', 'o01', 'o02', 'o03']].mean()

mean['name'] = mean.index
mean.reset_index(drop=True, inplace=True)

1-2. 역들의 좌표

지도에 표시하려면 좌표가 필요한데, 지하철역의 좌표를 정리해놓은 공공데이터는 찾을 수 없었다. 

https://observablehq.com/@taekie/seoul_subway_station_coordinate
따라서 해당 사이트에서 정리해놓은 자료를 사용하였다.

In [ ]:
locx2 = path+"\\1_columnchanged\coord.csv"

x2 = pd.read_csv(locx2, encoding='utf-8')
df2 = pd.DataFrame(x2)

1-3. 이용인원 + 좌표 합친 새로운 dataframe

name column을 기준으로 merge할 것인데, 이름이 일치하지 않는것이 있을것이다. 따라서, 일단 이름을 비교하고, 원본 파일을 수정해서 다시 불러오겠다.

In [ ]:
name1 = mean['name'].tolist()
name2 = df2['name'].tolist()

name1.sort()
name2.sort()

for x in name1:
    print(x)
print("\n\n")
for x in name2:
    print(x)

여기까지 진행하니 무언가 단단히 잘못됨을 느꼈다. 두 파일을 merge하려니 죄다 어긋낫다. 

2. 파일 수정

두 파일 모두 많이 수정을 해야한다.

작업을 하다보니 csv파일로는 자꾸 엑셀이 뻗어서, 이들을 전부 먼저 xlsx파일로 변환한뒤 작업하였다.

- 첫번째 파일에서
    'line'의 9호선 2~3단계 -> 9호선
    'line'의 9호선 2단계 -> 9호선
    
- 두번째 파일에서
    0*호선 -> *호선

- 첫번째, 두번째 파일에서
    노선명이 다르면 통일

3. 데이터프레임으로 파일 처리

- 첫번째 파일, 두번째 파일에서 
    공통노선 구한 뒤 없는 노선 삭제
    
를 2.의 엑셀상에서 하려 했으나, 데이터가 크다보니 엑셀이 자꾸 뻗었다.

이를 pandas로 불러와서 데이터 프레임으로 작업하겠다.

In [ ]:
locx3 = path+"\\2_valuefixed\metro2312.xlsx"

x3 = pd.read_excel(locx3)
df3 = pd.DataFrame(x3)

In [ ]:
locx4 = path+"\\2_valuefixed\coord.xlsx"

x4 = pd.read_excel(locx4)
df4 = pd.DataFrame(x4)

x3에는 숭실대학교, 숭실대학교(살피재) 처럼 역 이름이 갈라진 경우가 잦다.

정규표현식을 사용하여 이를 없애겠다.

In [ ]:
df3['name'] = df3['name'].str.replace(r'\(.*\)', '', regex=True)

x3를 name, line column을 기준으로 그룹짓겠다.

In [ ]:
gdf3 = df3.groupby(['name', 'line'])

# data 원본은 승차, 하차, 승차, 하차, ...
# 다루기 편하게 승차 column(i**), 하차 column(o**) 나눠놓음
newmean = gdf3[['i04', 'i05', 'i06', 'i07', 'i08', 'i09', 'i10', 'i11', 'i12', 'i13', 'i14', 'i15', 'i16', 'i17', 'i18', 'i19', 'i20', 'i21', 'i22', 'i23', 'i00', 'i01', 'i02', 'i03', 'o04', 'o05', 'o06', 'o07', 'o08', 'o09', 'o10', 'o11', 'o12', 'o13', 'o14', 'o15', 'o16', 'o17', 'o18', 'o19', 'o20', 'o21', 'o22', 'o23', 'o00', 'o01', 'o02', 'o03']].mean()

In [ ]:
newmean['nameline'] = newmean.index
newmean.reset_index(drop=True, inplace=True)

merge 하기위해, 기준으로 잡을 'nameline'을 df4에도 새로 만들겠다.

In [ ]:
df4['nameline'] = list(zip(df4['name'], df4['line']))

드디어 merge하겠다. inner방식을 써서, 중복되지 않는 'nameline'이 있으면 전부 날리겠다.

In [ ]:
mdf = pd.merge(df4, newmean, on='nameline', how='inner')
mdf

출력결과, mdf에서 중간에 좌표가 없는 row들이 몇 개 있다.

In [ ]:
mdf.to_excel(path+"\\3_onefile"+"\onefile.xlsx", index=False)

mdf 전부를 새로운 xlsx 파일로 "3_onefile" 디렉토리에 export해주었다. 

다음 과정부터, 새로 나온 유의미한 xlsx 파일만 있으면 되기에, 연산이 아주 가벼워진다.
numpy의 강력함을 새삼 느낀다. 

export 된 파일을 보니, 어떤 지하철역은 모든시간대에 타고 내리는 사람이 0명에 수렴하는 역도 있었다.
이가 outlier인 줄 알았으나 데이터를 더 살펴보니, 두개 이상 노선이 지나가는 역에서 어떤 한 호선에 승차하는 인원이 잘 없는것이였다.

어떤 지하철 역에서 타고 내리는게 편하다는것이 열차 내에서 편하게 갈 수 있다는 뜻이 아니다!!! 

('미사', '5호선')
('하남풍산', '5호선')
('석촌고분', '9호선')
('송파나루', '9호선')
('한성백제', '9호선')
('둔촌오륜', '9호선')
('삼전', '9호선')
('중앙보훈병원', '9호선')
('임진강', '경의선')
('인천공항1터미널', '공항철도')
('인천공항2터미널', '공항철도')

onefile.xlsx에서 좌표가 없는 역들은 위와 같다. 

이를 수동으로 채워주어 "4_filefinal" 디렉토리에 metrowithcoord.xlsx로 저장 후 필요시에 불러오겠다.


4. 수치 계산 (1)

In [ ]:
locx5 = path+"\\4_filefullfilled\metrowithcoord.xlsx"

x5 = pd.read_excel(locx5)
df5 = pd.DataFrame(x5)

역의 상대적인 규모를 계산해보자.

In [ ]:
df5['sum'] = df5[['i04', 'i05', 'i06', 'i07', 'i08', 'i09', 'i10', 'i11', 'i12', 'i13', 'i14', 'i15', 'i16', 'i17', 'i18', 'i19', 'i20', 'i21', 'i22', 'i23', 'i00', 'i01', 'i02', 'i03', 'o04', 'o05', 'o06', 'o07', 'o08', 'o09', 'o10', 'o11', 'o12', 'o13', 'o14', 'o15', 'o16', 'o17', 'o18', 'o19', 'o20', 'o21', 'o22', 'o23', 'o00', 'o01', 'o02', 'o03']].sum(axis=1)
max_val = df5['sum'].max()

df5['percentile'] = (df5['sum']/max_val)*100

'sum'은 일일 이용자 수 합의 평균이다.
'percentile'은 절대적인 이용자 수로 비교한 역들의 상대적인 수용규모이다.

In [ ]:
df5['imax'] = df5[['i04', 'i05', 'i06', 'i07', 'i08', 'i09', 'i10', 'i11', 'i12', 'i13', 'i14', 'i15', 'i16', 'i17', 'i18', 'i19', 'i20', 'i21', 'i22', 'i23', 'i00', 'i01', 'i02', 'i03']].max(axis=1)
df5['omax'] = df5[['o04', 'o05', 'o06', 'o07', 'o08', 'o09', 'o10', 'o11', 'o12', 'o13', 'o14', 'o15', 'o16', 'o17', 'o18', 'o19', 'o20', 'o21', 'o22', 'o23', 'o00', 'o01', 'o02', 'o03']].max(axis=1)

df5['io04'] = df5[['i04', 'o04']].sum(axis=1)
df5['io05'] = df5[['i05', 'o05']].sum(axis=1)
df5['io06'] = df5[['i06', 'o06']].sum(axis=1)
df5['io07'] = df5[['i07', 'o07']].sum(axis=1)
df5['io08'] = df5[['i08', 'o08']].sum(axis=1)
df5['io09'] = df5[['i09', 'o09']].sum(axis=1)
df5['io10'] = df5[['i10', 'o10']].sum(axis=1)
df5['io11'] = df5[['i11', 'o11']].sum(axis=1)
df5['io12'] = df5[['i12', 'o12']].sum(axis=1)
df5['io13'] = df5[['i13', 'o13']].sum(axis=1)
df5['io14'] = df5[['i14', 'o14']].sum(axis=1)
df5['io15'] = df5[['i15', 'o15']].sum(axis=1)
df5['io16'] = df5[['i16', 'o16']].sum(axis=1)
df5['io17'] = df5[['i17', 'o17']].sum(axis=1)
df5['io18'] = df5[['i18', 'o18']].sum(axis=1)
df5['io19'] = df5[['i19', 'o19']].sum(axis=1)
df5['io20'] = df5[['i20', 'o20']].sum(axis=1)
df5['io21'] = df5[['i21', 'o21']].sum(axis=1)
df5['io22'] = df5[['i22', 'o22']].sum(axis=1)
df5['io23'] = df5[['i23', 'o23']].sum(axis=1)
df5['io00'] = df5[['i00', 'o00']].sum(axis=1)
df5['io01'] = df5[['i01', 'o01']].sum(axis=1)
df5['io02'] = df5[['i02', 'o02']].sum(axis=1)
df5['io03'] = df5[['i03', 'o03']].sum(axis=1)

df5['iomax'] = df5[['io04', 'io05', 'io06', 'io07', 'io08', 'io09', 'io10', 'io11', 'io12', 'io13', 'io14', 'io15', 'io16', 'io17', 'io18', 'io19', 'io20', 'io21', 'io22', 'io23', 'io00', 'io01', 'io02', 'io03']].max(axis=1)

'imax'는 어떤 역의 최대 승차 시간의 승차 인원수이다.
'omax'는 어떤 역의 최대 하차 시간의 하차 인원수이다.
'iomax'는 어떤 역이 가장 붐비는(i** + o**) 시간이다.

In [ ]:
complexdf = df5[['line','name','lat','lng', 'percentile']].copy()

complexdf['ci04'] = df5['i04'] / df5['imax'] * 100
complexdf['ci05'] = df5['i05'] / df5['imax'] * 100
complexdf['ci06'] = df5['i06'] / df5['imax'] * 100
complexdf['ci07'] = df5['i07'] / df5['imax'] * 100
complexdf['ci08'] = df5['i08'] / df5['imax'] * 100
complexdf['ci09'] = df5['i09'] / df5['imax'] * 100
complexdf['ci10'] = df5['i10'] / df5['imax'] * 100
complexdf['ci11'] = df5['i11'] / df5['imax'] * 100
complexdf['ci12'] = df5['i12'] / df5['imax'] * 100
complexdf['ci13'] = df5['i13'] / df5['imax'] * 100
complexdf['ci14'] = df5['i14'] / df5['imax'] * 100
complexdf['ci15'] = df5['i15'] / df5['imax'] * 100
complexdf['ci16'] = df5['i16'] / df5['imax'] * 100
complexdf['ci17'] = df5['i17'] / df5['imax'] * 100
complexdf['ci18'] = df5['i18'] / df5['imax'] * 100
complexdf['ci19'] = df5['i19'] / df5['imax'] * 100
complexdf['ci20'] = df5['i20'] / df5['imax'] * 100
complexdf['ci21'] = df5['i21'] / df5['imax'] * 100
complexdf['ci22'] = df5['i22'] / df5['imax'] * 100
complexdf['ci23'] = df5['i23'] / df5['imax'] * 100
complexdf['ci00'] = df5['i00'] / df5['imax'] * 100
complexdf['ci01'] = df5['i01'] / df5['imax'] * 100
complexdf['ci02'] = df5['i02'] / df5['imax'] * 100
complexdf['ci03'] = df5['i03'] / df5['imax'] * 100

complexdf['co04'] = df5['o04'] / df5['omax'] * 100
complexdf['co05'] = df5['o05'] / df5['omax'] * 100
complexdf['co06'] = df5['o06'] / df5['omax'] * 100
complexdf['co07'] = df5['o07'] / df5['omax'] * 100
complexdf['co08'] = df5['o08'] / df5['omax'] * 100
complexdf['co09'] = df5['o09'] / df5['omax'] * 100
complexdf['co10'] = df5['o10'] / df5['omax'] * 100
complexdf['co11'] = df5['o11'] / df5['omax'] * 100
complexdf['co12'] = df5['o12'] / df5['omax'] * 100
complexdf['co13'] = df5['o13'] / df5['omax'] * 100
complexdf['co14'] = df5['o14'] / df5['omax'] * 100
complexdf['co15'] = df5['o15'] / df5['omax'] * 100
complexdf['co16'] = df5['o16'] / df5['omax'] * 100
complexdf['co17'] = df5['o17'] / df5['omax'] * 100
complexdf['co18'] = df5['o18'] / df5['omax'] * 100
complexdf['co19'] = df5['o19'] / df5['omax'] * 100
complexdf['co20'] = df5['o20'] / df5['omax'] * 100
complexdf['co21'] = df5['o21'] / df5['omax'] * 100
complexdf['co22'] = df5['o22'] / df5['omax'] * 100
complexdf['co23'] = df5['o23'] / df5['omax'] * 100
complexdf['co00'] = df5['o00'] / df5['omax'] * 100
complexdf['co01'] = df5['o01'] / df5['omax'] * 100
complexdf['co02'] = df5['o02'] / df5['omax'] * 100
complexdf['co03'] = df5['o03'] / df5['omax'] * 100

complexdf['cio04'] = df5['io04'] / df5['iomax'] * 100
complexdf['cio05'] = df5['io05'] / df5['iomax'] * 100
complexdf['cio06'] = df5['io06'] / df5['iomax'] * 100
complexdf['cio07'] = df5['io07'] / df5['iomax'] * 100
complexdf['cio08'] = df5['io08'] / df5['iomax'] * 100
complexdf['cio09'] = df5['io09'] / df5['iomax'] * 100
complexdf['cio10'] = df5['io10'] / df5['iomax'] * 100
complexdf['cio11'] = df5['io11'] / df5['iomax'] * 100
complexdf['cio12'] = df5['io12'] / df5['iomax'] * 100
complexdf['cio13'] = df5['io13'] / df5['iomax'] * 100
complexdf['cio14'] = df5['io14'] / df5['iomax'] * 100
complexdf['cio15'] = df5['io15'] / df5['iomax'] * 100
complexdf['cio16'] = df5['io16'] / df5['iomax'] * 100
complexdf['cio17'] = df5['io17'] / df5['iomax'] * 100
complexdf['cio18'] = df5['io18'] / df5['iomax'] * 100
complexdf['cio19'] = df5['io19'] / df5['iomax'] * 100
complexdf['cio20'] = df5['io20'] / df5['iomax'] * 100
complexdf['cio21'] = df5['io21'] / df5['iomax'] * 100
complexdf['cio22'] = df5['io22'] / df5['iomax'] * 100
complexdf['cio23'] = df5['io23'] / df5['iomax'] * 100
complexdf['cio00'] = df5['io00'] / df5['iomax'] * 100
complexdf['cio01'] = df5['io01'] / df5['iomax'] * 100
complexdf['cio02'] = df5['io02'] / df5['iomax'] * 100
complexdf['cio03'] = df5['io03'] / df5['iomax'] * 100

'ci'는 승차 자체의 복잡도이다
'co'는 하자 자체의 복잡도이다
'cio'는 역의 복잡도이다

In [ ]:
complexdf.to_excel(path+"\\5_complex"+"\complex.xlsx", index=False)

어느정도 진행 후, 주피터 노트북을 가볍게 쓰기 위해 complexdf 전부를 새로운 xlsx 파일로 "5_complex" 디렉토리에 export해주었다. 

이 파일이 이제 실질적인 결과 도출에 쓰일것이다.

5. 수치 계산 (2)

In [ ]:
locx6 = path+"\\5_complex\complex.xlsx"

x6 = pd.read_excel(locx6)
df6 = pd.DataFrame(x6)

In [ ]:
complexdf2 = df6[['line','name','lat','lng', 'percentile']].copy()

complexdf2['cci04'] = ( df6['ci04'] + df6['cio04'] )/2 * df6['percentile']
complexdf2['cci05'] = ( df6['ci05'] + df6['cio05'] )/2 * df6['percentile']
complexdf2['cci06'] = ( df6['ci06'] + df6['cio06'] )/2 * df6['percentile']
complexdf2['cci07'] = ( df6['ci07'] + df6['cio07'] )/2 * df6['percentile']
complexdf2['cci08'] = ( df6['ci08'] + df6['cio08'] )/2 * df6['percentile']
complexdf2['cci09'] = ( df6['ci09'] + df6['cio09'] )/2 * df6['percentile']
complexdf2['cci10'] = ( df6['ci10'] + df6['cio10'] )/2 * df6['percentile']
complexdf2['cci11'] = ( df6['ci11'] + df6['cio11'] )/2 * df6['percentile']
complexdf2['cci12'] = ( df6['ci12'] + df6['cio12'] )/2 * df6['percentile']
complexdf2['cci13'] = ( df6['ci13'] + df6['cio13'] )/2 * df6['percentile']
complexdf2['cci14'] = ( df6['ci14'] + df6['cio14'] )/2 * df6['percentile']
complexdf2['cci15'] = ( df6['ci15'] + df6['cio15'] )/2 * df6['percentile']
complexdf2['cci16'] = ( df6['ci16'] + df6['cio16'] )/2 * df6['percentile']
complexdf2['cci17'] = ( df6['ci17'] + df6['cio17'] )/2 * df6['percentile']
complexdf2['cci18'] = ( df6['ci18'] + df6['cio18'] )/2 * df6['percentile']
complexdf2['cci19'] = ( df6['ci19'] + df6['cio19'] )/2 * df6['percentile']
complexdf2['cci20'] = ( df6['ci20'] + df6['cio20'] )/2 * df6['percentile']
complexdf2['cci21'] = ( df6['ci21'] + df6['cio21'] )/2 * df6['percentile']
complexdf2['cci22'] = ( df6['ci22'] + df6['cio22'] )/2 * df6['percentile']
complexdf2['cci23'] = ( df6['ci23'] + df6['cio23'] )/2 * df6['percentile']
complexdf2['cci00'] = ( df6['ci00'] + df6['cio00'] )/2 * df6['percentile']
complexdf2['cci01'] = ( df6['ci01'] + df6['cio01'] )/2 * df6['percentile']
complexdf2['cci02'] = ( df6['ci02'] + df6['cio02'] )/2 * df6['percentile']
complexdf2['cci03'] = ( df6['ci03'] + df6['cio03'] )/2 * df6['percentile']

complexdf2['cco04'] = ( df6['co04'] + df6['cio04'] )/2 * df6['percentile']
complexdf2['cco05'] = ( df6['co05'] + df6['cio05'] )/2 * df6['percentile']
complexdf2['cco06'] = ( df6['co06'] + df6['cio06'] )/2 * df6['percentile']
complexdf2['cco07'] = ( df6['co07'] + df6['cio07'] )/2 * df6['percentile']
complexdf2['cco08'] = ( df6['co08'] + df6['cio08'] )/2 * df6['percentile']
complexdf2['cco09'] = ( df6['co09'] + df6['cio09'] )/2 * df6['percentile']
complexdf2['cco10'] = ( df6['co10'] + df6['cio10'] )/2 * df6['percentile']
complexdf2['cco11'] = ( df6['co11'] + df6['cio11'] )/2 * df6['percentile']
complexdf2['cco12'] = ( df6['co12'] + df6['cio12'] )/2 * df6['percentile']
complexdf2['cco13'] = ( df6['co13'] + df6['cio13'] )/2 * df6['percentile']
complexdf2['cco14'] = ( df6['co14'] + df6['cio14'] )/2 * df6['percentile']
complexdf2['cco15'] = ( df6['co15'] + df6['cio15'] )/2 * df6['percentile']
complexdf2['cco16'] = ( df6['co16'] + df6['cio16'] )/2 * df6['percentile']
complexdf2['cco17'] = ( df6['co17'] + df6['cio17'] )/2 * df6['percentile']
complexdf2['cco18'] = ( df6['co18'] + df6['cio18'] )/2 * df6['percentile']
complexdf2['cco19'] = ( df6['co19'] + df6['cio19'] )/2 * df6['percentile']
complexdf2['cco20'] = ( df6['co20'] + df6['cio20'] )/2 * df6['percentile']
complexdf2['cco21'] = ( df6['co21'] + df6['cio21'] )/2 * df6['percentile']
complexdf2['cco22'] = ( df6['co22'] + df6['cio22'] )/2 * df6['percentile']
complexdf2['cco23'] = ( df6['co23'] + df6['cio23'] )/2 * df6['percentile']
complexdf2['cco00'] = ( df6['co00'] + df6['cio00'] )/2 * df6['percentile']
complexdf2['cco01'] = ( df6['co01'] + df6['cio01'] )/2 * df6['percentile']
complexdf2['cco02'] = ( df6['co02'] + df6['cio02'] )/2 * df6['percentile']
complexdf2['cco03'] = ( df6['co03'] + df6['cio03'] )/2 * df6['percentile']

In [ ]:
complexdfi = complexdf2[['cci04', 'cci05', 'cci06', 'cci07', 'cci08', 'cci09', 'cci10', 'cci11', 'cci12', 'cci13', 'cci14', 'cci15', 'cci16', 'cci17', 'cci18', 'cci19', 'cci20', 'cci21', 'cci22', 'cci23', 'cci00', 'cci01', 'cci02', 'cci03']].copy()

complexdfo = complexdf2[['cco04', 'cco05', 'cco06', 'cco07', 'cco08', 'cco09', 'cco10', 'cco11', 'cco12', 'cco13', 'cco14', 'cco15', 'cco16', 'cco17', 'cco18', 'cco19', 'cco20', 'cco21', 'cco22', 'cco23', 'cco00', 'cco01', 'cco02', 'cco03']].copy()

listi = complexdfi.values.tolist()
listo = complexdfo.values.tolist()

arri = np.array(listi)
arri = np.nan_to_num(arri, nan=0)
arro = np.array(listo)
arro = np.nan_to_num(arro, nan=0)

In [ ]:
meani = np.mean(arri)
meano = np.mean(arro)

stdi = np.std(arri)
stdo = np.std(arro)

narri = 100 * (1 / (1 + np.exp(-(arri - meani) / stdi)))
narro = 100 * (1 / (1 + np.exp(-(arro - meano) / stdo)))

In [ ]:
coli = ['i04', 'i05', 'i06', 'i07', 'i08', 'i09', 'i10', 'i11', 'i12', 'i13', 'i14', 'i15', 'i16', 'i17', 'i18', 'i19', 'i20', 'i21', 'i22', 'i23', 'i00', 'i01', 'i02', 'i03']
colo = ['o04', 'o05', 'o06', 'o07', 'o08', 'o09', 'o10', 'o11', 'o12', 'o13', 'o14', 'o15', 'o16', 'o17', 'o18', 'o19', 'o20', 'o21', 'o22', 'o23', 'o00', 'o01', 'o02', 'o03']

dfni = pd.DataFrame(narri, columns=coli)
dfno = pd.DataFrame(narro, columns=colo)

In [ ]:
dfnio = pd.concat([dfni, dfno], axis=1)

complexdfn = df6[['line', 'name', 'lat', 'lng', 'percentile']].copy()

complexdfn = pd.concat([complexdfn, dfnio], axis=1)

complexdfn.to_excel(path+"\\6_ncomplex"+"\\ncomplex.xlsx", index=False)

평준화한 복잡도를 엑셀로 저장하였다. 이제 Folium에서 이를 불러와서 사용할 것이다.

6. Foilum으로 시각화

In [ ]:
locx7 = path+"\\6_ncomplex\\ncomplex.xlsx"

x7 = pd.read_excel(locx7)
df7 = pd.DataFrame(x7)

In [4]:
def whatcolor(val):
    if(val<40):
        return 'purple'
    elif(val<50):
        return 'blue'
    elif(val<60):
        return 'green'
    elif(val<70):
        return 'orange'
    elif(val<80):
        return 'red'
    elif(val<90):
        return 'gray'
    elif(val<100):
        return 'black'

t = 4
io = 'o'

while(1):
    if t > 23:
        t = t-24
    
    if t == 4:
        if io == 'i':
            io = 'o'
        elif io == 'o':
            io = 'i'
            
    iot = io+str(t).zfill(2)
    print(iot)
    
    m = folium.Map(location=[37.566352,126.977952], zoom_start=12)
    
    for index, row in df7.iterrows():
        ncom = row[iot]  # 혼잡도
        wc = whatcolor(ncom)  # 혼잡도로 얻은 아이콘 색깔
        folium.Marker(location=[row['lat'], row['lng']], popup=ncom, tooltip=row['name'],  icon=folium.Icon(color=wc) ).add_to(m)
    
    dp.display(m)
    time.sleep(2)
    dp.clear_output(wait=True)
    t = t+1

i09


KeyboardInterrupt: 